In [3]:
import pandas as pd
import numpy as np


In [6]:
df_cliente = pd.read_csv('/content/drive/MyDrive/VAI/Projeto Final/Dados/cliente.csv')
df_produto = pd.read_csv('/content/drive/MyDrive/VAI/Projeto Final/Dados/produto.csv')
df_venda = pd.read_csv('/content/drive/MyDrive/VAI/Projeto Final/Dados/venda.csv')

<ipython-input-6-d6e0fe299ed8>:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_venda = pd.read_csv('/content/drive/MyDrive/VAI/Projeto Final/Dados/venda.csv')


In [7]:
df_cliente

,ID_CLIENTE,SEXO,DATA_NASCIMENTO,PRIMEIRA_COMPRA
0,0,,1900/01/01,2017/03/06
1,1,,1900/01/01,2017/03/06
2,2,,1900/01/01,2017/03/06
3,3,,1900/01/01,2017/02/18
4,4,,1900/01/01,2017/01/16
...,...,...,...,...
105919,105919,F,1977/09/11,2021/10/16
105920,105920,,1900/01/01,1900/01/01
105921,105921,,1900/01/01,1900/01/01
105922,105922,F,1985/05/15,2021/10/16


In [10]:
df_cliente['SEXO'].value_counts(dropna=False)

     59401
F    26809
M    17384
O     1307
f      575
m      448
Name: SEXO, dtype: int64

In [18]:
df_cliente['DATA_NASCIMENTO'].value_counts(dropna=False)

1900/01/01    60579
2020/08/13       43
2020/08/15       37
2020/08/14       32
2000/01/01       27
              ...  
1983/07/12        1
1973/03/12        1
1947/11/24        1
2007/11/07        1
1962/12/16        1
Name: DATA_NASCIMENTO, Length: 16377, dtype: int64

In [19]:
df_cliente['PRIMEIRA_COMPRA'].value_counts(dropna=False)

1900/01/01    4060
2020/11/14     319
2020/12/12     274
2020/11/21     270
2020/11/28     267
              ... 
2017/03/20       1
2019/02/10       1
2019/05/29       1
2017/01/16       1
2017/01/30       1
Name: PRIMEIRA_COMPRA, Length: 1693, dtype: int64

In [20]:
df_cliente[df_cliente['DATA_NASCIMENTO'] == df_cliente['PRIMEIRA_COMPRA']]

,ID_CLIENTE,SEXO,DATA_NASCIMENTO,PRIMEIRA_COMPRA
63,63,,1900/01/01,1900/01/01
92,92,,1900/01/01,1900/01/01
133,133,,1900/01/01,1900/01/01
151,151,,1900/01/01,1900/01/01
173,173,,1900/01/01,1900/01/01
...,...,...,...,...
105803,105803,,1900/01/01,1900/01/01
105830,105830,,1900/01/01,1900/01/01
105852,105852,,1900/01/01,1900/01/01
105920,105920,,1900/01/01,1900/01/01


In [5]:
df_venda

,ID_VENDA,DATA_VENDA,ID_CLIENTE,ID_PRODUTO,QUANTIDADE,CATEGORIA_VENDA,FILIAL
0,0,2021/02/10,45659.0,17631,1,E-COMMERCE,101
1,0,2021/02/10,45659.0,34575,1,E-COMMERCE,101
2,1,2021/02/12,70766.0,2240,1,E-COMMERCE,101
3,2,2021/02/12,42579.0,17707,1,E-COMMERCE,101
4,2,2021/02/12,42579.0,8952,1,E-COMMERCE,101
...,...,...,...,...,...,...,...
1859578,854530,2021/10/23,NaN,36642,10.0,Loja,113
1859579,854530,2021/10/23,NaN,7332,10.0,Loja,113
1859580,854530,2021/10/23,NaN,7333,10.0,Loja,113
1859581,854531,2021/10/23,106865.0,7528,1.0,Loja,113


In [ ]:
df_produto

,ID_PRODUTO,UNIDADE,NIVEL 1,NIVEL 2,NIVEL 3,NIVEL 4
0,0,UN,CAES,ALIMENTOS,PETISCOS E OSSINHOS,BISCOITOS - BIFINHOS - SNACKS
1,1,UN,CAES,ALIMENTOS,PETISCOS E OSSINHOS,BISCOITOS - BIFINHOS - SNACKS
2,2,UN,CAES,ALIMENTOS,PETISCOS E OSSINHOS,BISCOITOS - BIFINHOS - SNACKS
3,3,UN,,,,
4,4,UN,CAES,FARMACIA,DORES E INFECCOES,ANTIBIOTICOS
...,...,...,...,...,...,...
38303,38303,UN,,,,
38304,38304,UN,,,,
38305,38305,UN,,,,
38306,38306,UN,,,,
